In [352]:
import json
import re

In [353]:
#TODO transformar para CNF
#TODO quebrar em cenários OR
#TODO TRATAR CASO DA IGUALDADE "a=3".

In [354]:


def parse_cnf_expression(expr):
    """Parses a CNF expression into a list of dictionaries with variable conditions."""
    # Remove all spaces from the expression
    expr = expr.replace(" ", "")
    clauses = expr.split('AND')
    parsed_clauses = []

    for clause in clauses:
        # Regex to match the variable and its condition
        match = re.match(r'(\w+)([<>=]+)(-?\d+)', clause.strip())
        if match:
            var = match.group(1)
            operator = match.group(2)
            value = match.group(3)
            condition = f"{operator}{value}"
            parsed_clauses.append({var: {condition}})

    return parsed_clauses

In [355]:
def intersect_cnf(parsed_expr1, parsed_expr2):
    """Finds the intersection of variable sets from two CNF expressions."""

    # Convert parsed expressions to dictionaries for easy lookup
    dict_expr1 = {list(d.keys())[0]: list(d.values())[0] for d in parsed_expr1}
    dict_expr2 = {list(d.keys())[0]: list(d.values())[0] for d in parsed_expr2}

    intersection_expr1 = []
    intersection_expr2 = []

    for var in dict_expr1:
        if var in dict_expr2:
            # Keep only the conditions that are in both expressions
            intersection_expr1.append({var: dict_expr1[var]})
            intersection_expr2.append({var: dict_expr2[var]})

    return intersection_expr1, intersection_expr2

In [356]:


def parse_intervals(preprocessed_expr, variable_limits):
    """Converts a preprocessed CNF expression into a list of intervals."""
    intervals = {}
    for var in variable_limits:
        intervals[var] = {'min': variable_limits[var][0], 'max': variable_limits[var][1]}

    for clause in preprocessed_expr:
        for var, condition_set in clause.items():
            for condition in condition_set:
                operator, value = re.match(r'([<>=]+)(-?\d+)', condition).groups()
                value = int(value)
                if operator == '>=':
                    intervals[var]['min'] = max(intervals[var]['min'], value)
                elif operator == '<=':
                    intervals[var]['max'] = min(intervals[var]['max'], value)
                elif operator == '>':
                    intervals[var]['min'] = max(intervals[var]['min'], value + 1)
                elif operator == '<':
                    intervals[var]['max'] = min(intervals[var]['max'], value - 1)
                elif operator == '=':
                    intervals[var]['min'] = intervals[var]['max'] = value
    return [(var, v['min'], v['max']) for var, v in intervals.items()]

def calculate_volume(intervals):
    """Calculates the volume of the n-dimensional region defined by the intervals."""
    volume = 1
    for _, min_val, max_val in intervals:
        if min_val > max_val:
            return 0
        volume *= (max_val - min_val)
    return volume

def calculate_intersection_volume(intervals1, intervals2):
    """Calculates the volume of the intersection of two n-dimensional regions."""
    intersection_volume = 1
    for (var1, min1, max1), (var2, min2, max2) in zip(intervals1, intervals2):
        if var1 != var2:
            raise ValueError("Mismatched variables in intervals.")
        intersection_min = max(min1, min2)
        intersection_max = min(max1, max2)
        if intersection_min > intersection_max:
            return 0  # No intersection
        intersection_volume *= (intersection_max - intersection_min)
    return intersection_volume

def jaccard_similarity(preprocessed_expr1, preprocessed_expr2, variable_limits):
    intervals1 = parse_intervals(preprocessed_expr1, variable_limits)
    intervals2 = parse_intervals(preprocessed_expr2, variable_limits)

    volume1 = calculate_volume(intervals1)
    volume2 = calculate_volume(intervals2)
    intersection_volume = calculate_intersection_volume(intervals1, intervals2)
    union_volume = volume1 + volume2 - intersection_volume

    if union_volume == 0:
        return 0  # To avoid division by zero

    similarity = intersection_volume / union_volume
    return similarity

In [357]:
def tversky_similarity(preprocessed_expr1, preprocessed_expr2, alpha=1, beta=1):

    """Calculates the Tversky similarity between two preprocessed CNF expressions."""
    dict_expr1 = {list(d.keys())[0]: list(d.values())[0] for d in preprocessed_expr1}
    dict_expr2 = {list(d.keys())[0]: list(d.values())[0] for d in preprocessed_expr2}

    intersection_size = 0
    diff1_size = 0
    diff2_size = 0

    all_vars = set(dict_expr1.keys()).union(set(dict_expr2.keys()))

    for var in all_vars:
        if var in dict_expr1 and var in dict_expr2:
            intersection_size += 1
        elif var in dict_expr1:
            diff1_size += 1
        elif var in dict_expr2:
            diff2_size += 1

    similarity = intersection_size / (intersection_size + alpha * diff1_size + beta * diff2_size)
    return similarity

In [358]:

def get_variables_limits(file_path = 'variables_config.json'):
    #Definindo os limites conhecidos para as variáveis

    # Ler o arquivo JSON
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Construir o dicionário de tuplas
    variable_limits = {item["name"]: (item["min_value"], item["max_value"]) for item in data}

    # Exibir o dicionário
    return variable_limits

In [359]:
def get_desired_scenario(file_path = 'analyzed_scenario_process.json'):
    # Ler o arquivo JSON
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Pegar o valor da chave 'desired'
    desired_scenario = data.get('desired')
    # desired_scenario_name = next(iter(desired_scenario))
    # Exibir o valor
    return desired_scenario

In [360]:
def get_precondition(scenario):
    scenario_name = next(iter(scenario))

    given_exp=scenario[scenario_name]["Given"]
    when_exp=scenario[scenario_name]["When"]

    precond = "{} AND {}".format(given_exp, when_exp)
    print(f"Precondition: {precond}")
    return precond

In [361]:
def get_postcondition(scenario):

    scenario_name = next(iter(scenario))
    postcond =  scenario[scenario_name]["Then"]
    print(f"Postcondition: {postcond}")
    return postcond

In [362]:
def similarity_between_expressions(expression1, expression2, variable_limits):
    parsed_expression1 = parse_cnf_expression(expression1)
    print("parsed_expression 1: {}".format(parsed_expression1))

    parsed_expression2 = parse_cnf_expression(expression2)
    print("parsed_expression 2: {}".format(parsed_expression2))

    intersected_expr1, intersected_expr2 = intersect_cnf(parsed_expression1, parsed_expression2)
    print("intersected_expr 1: {}".format(intersected_expr1))
    print("intersected_expr 2: {}".format(intersected_expr2))

    # Similaridade de condição
    jaccard_sim = jaccard_similarity(intersected_expr1, intersected_expr2, variable_limits)
    # print(f"Jaccard Similarity: {jaccard_sim:.3f}")

    # Similaridade de variáveis
    tversky_sim=tversky_similarity(parsed_expression1, parsed_expression2, alpha=1, beta=1)
    # print(f"Tversky Similarity: {tversky_sim:.3f}")

    return jaccard_sim, tversky_sim

In [363]:
def similarity_between_scenarios(scenario_a, scenario_b, variable_limits):
    expression_precond_scenario_a = get_precondition(scenario_a)
    expression_precond_scenario_b = get_precondition(scenario_b)

    jaccard_sim_precond, tversky_sim_precond = similarity_between_expressions(expression_precond_scenario_a, expression_precond_scenario_b, variable_limits)

    expression_postcond_scenario_a = get_postcondition(scenario_a)
    expression_postcond_scenario_b = get_postcondition(scenario_b)

    jaccard_sim_postcond, tversky_sim_postcond = similarity_between_expressions(expression_postcond_scenario_a, expression_postcond_scenario_b, variable_limits)

    return jaccard_sim_precond, tversky_sim_precond, jaccard_sim_postcond, tversky_sim_postcond

In [364]:
variable_limits=get_variables_limits()
print(variable_limits)

desired_scenario=get_desired_scenario()
print(desired_scenario)

# print(desired_scenario["Given"])
# print(desired_scenario["When"])
# print(desired_scenario["Then"])



{'height': (0, 100), 'distance_target': (0.0, 100.0), 'obstacle': (0, 100)}
{'Landing': {'Given': 'height > 3 AND obstacle > 10', 'When': 'distance_target <=1', 'Do': 'similar.Do', 'Then': 'height < 10 AND distance_target <=5'}}


In [365]:
# Caminho do arquivo JSON
file_path = 'shared_base.json'

# Ler o arquivo JSON
with open(file_path, 'r') as file:
    shared_scenarios = json.load(file)

similarity_result = {}
for shared_scenario in shared_scenarios:
    jaccard_sim_precond, tversky_sim_precond, jaccard_sim_postcond, tversky_sim_postcond = similarity_between_scenarios(desired_scenario, shared_scenario, variable_limits)

    print(f"Precondition - Jaccard Similarity: {jaccard_sim_precond}, Tversky Similarity: {tversky_sim_precond}")
    print(f"Postcondition - Jaccard Similarity: {jaccard_sim_postcond}, Tversky Similarity: {tversky_sim_postcond}")
    
    # scenario_name=next(iter(desired_scenario))
    
    # similarity_result[shared_scenario[scenario_name]] = {"jaccard":jaccard_sim, "tversky":tversky_sim}

Precondition: height > 3 AND obstacle > 10 AND distance_target <=1
Precondition: height > 3 AND obstacle < 10 AND distance_target <=5
parsed_expression 1: [{'height': {'>3'}}, {'obstacle': {'>10'}}, {'distance_target': {'<=1'}}]
parsed_expression 2: [{'height': {'>3'}}, {'obstacle': {'<10'}}, {'distance_target': {'<=5'}}]
intersected_expr 1: [{'height': {'>3'}}, {'obstacle': {'>10'}}, {'distance_target': {'<=1'}}]
intersected_expr 2: [{'height': {'>3'}}, {'obstacle': {'<10'}}, {'distance_target': {'<=5'}}]
Postcondition: height < 10 AND distance_target <=5
Postcondition: height < 10 AND distance_target <=5
parsed_expression 1: [{'height': {'<10'}}, {'distance_target': {'<=5'}}]
parsed_expression 2: [{'height': {'<10'}}, {'distance_target': {'<=5'}}]
intersected_expr 1: [{'height': {'<10'}}, {'distance_target': {'<=5'}}]
intersected_expr 2: [{'height': {'<10'}}, {'distance_target': {'<=5'}}]
Precondition - Jaccard Similarity: 0.0, Tversky Similarity: 1.0
Postcondition - Jaccard Similari

In [366]:
similarity_result

{}

In [367]:
###PRECONDITION

# Exemplo de uso
expr1 = "height > 3 AND obstacle > 10 AND distance_target <=1"

expr2 = "height > 3 AND obstacle < 10 AND distance_target <=5"



parsed_expression1 = parse_cnf_expression(expr1)
# print(parsed_expression1)

parsed_expression2 = parse_cnf_expression(expr2)
# print(parsed_expression2)

intersected_expr1, intersected_expr2 = intersect_cnf(parsed_expression1, parsed_expression2)
print("preprocessed 1: {}".format(intersected_expr1))
print("preprocessed 2: {}".format(intersected_expr2))

# Similaridade de condição
jaccard_sim1 = jaccard_similarity(intersected_expr1, intersected_expr2, variable_limits)
print(f"Jaccard Similarity: {jaccard_sim1:.3f}")

# Similaridade de variáveis
tversky_sim1=tversky_similarity(parsed_expression1, parsed_expression2, alpha=1, beta=1)
print(f"Tversky Similarity: {tversky_sim1:.3f}")

preprocessed 1: [{'height': {'>3'}}, {'obstacle': {'>10'}}, {'distance_target': {'<=1'}}]
preprocessed 2: [{'height': {'>3'}}, {'obstacle': {'<10'}}, {'distance_target': {'<=5'}}]
Jaccard Similarity: 0.000
Tversky Similarity: 1.000


In [368]:

print(jaccard_sim1*tversky_sim1)

print(jaccard_sim1-(1-tversky_sim1))

0.0
0.0


In [369]:
### eu gostei mais dessa formula
y=0.5
total_pre = y*jaccard_sim1+(1-y)*tversky_sim1
print(total_pre)

0.5


In [370]:
###POST CONDITION
# Definindo os limites conhecidos para as variáveis
variable_limits = {
    'a': (0, 100),
    'b': (0, 100),
    'c': (0, 100)
}

# Exemplo de uso
expr1 = "a>=3 AND b<=10 AND c>=10"

expr2 = "a>=3 AND b>=10 AND c>=10"

#TODO TRATAR CASO DA IGUALDADE "a=3".

parsed_expression1 = parse_cnf_expression(expr1)
# print(parsed_expression1)

parsed_expression2 = parse_cnf_expression(expr2)
# print(parsed_expression2)

intersected_expr1, intersected_expr2 = intersect_cnf(parsed_expression1, parsed_expression2)
print("preprocessed 1: {}".format(intersected_expr1))
print("preprocessed 2: {}".format(intersected_expr2))

# Similaridade de condição
jaccard_sim2 = jaccard_similarity(intersected_expr1, intersected_expr2, variable_limits)
print(f"Jaccard Similarity: {jaccard_sim2:.3f}")

# Similaridade de variáveis
tversky_sim2=tversky_similarity(parsed_expression1, parsed_expression2, alpha=1, beta=1)
print(f"Tversky Similarity: {tversky_sim2:.3f}")

preprocessed 1: [{'a': {'>=3'}}, {'b': {'<=10'}}, {'c': {'>=10'}}]
preprocessed 2: [{'a': {'>=3'}}, {'b': {'>=10'}}, {'c': {'>=10'}}]
Jaccard Similarity: 0.000
Tversky Similarity: 1.000


In [371]:
y=0.5
total_post = y*jaccard_sim2+(1-y)*tversky_sim2
print(total_post)

0.5


In [372]:
#tlz eu tenha que colocar uma coef para dar mais importancia ao pre ou ao post.(Avaliar)
sim_total = (total_pre+total_post)/2
sim_total

0.5